# Problem 1

Consider the problem of estimating 

$$\theta = \mathbb{P}[X > a]$$

where $X$ is some random variable and $a$ is a constant.  We will assume that $a$ is very large, so that $\theta$ is very small.  Define 

$$f_{X}(x; \beta) = \frac{e^{\beta x}f_{X}(x)}{Z(\beta)}$$

where 

$$Z(\beta) = \int_{-\infty}^{\infty}e^{\beta x}f_{X}(x)\,\textrm{d}x$$

and let $\mathbb{E}_{\beta}$ denote expectation with respect to the density $f_{X}(x; \beta)$.  (This is the same as our notation from class.)  You can assume that the integral in $Z(\beta)$ converges for any value of $\beta$.  

In class, we showed that 

$$\theta = \mathbb{E}_{\beta}\left[1_{(a, \infty)}(X)e^{-\beta X + \ln Z(\beta)}\right]$$

and we derived a formula for the variance of our importance sampling estimator in the form 

$$\textrm{Var}_{\beta}\left[1_{(a, \infty)}(X)e^{-\beta X + \ln Z(\beta)}\right] = F(\beta) - \theta^2$$

where

$$F(\beta) = \int_{-\infty}^{\infty}1_{(a, \infty)}(x)e^{-\beta x + \ln Z(\beta)}f_{X}(x)\,\textrm{d}x$$

(You should confirm to yourself that these match the general formulas we had in class, but there is no need to re-derive them on the homework.)

## Part 1

In order to minimize the variance of our importance sampling estimator, we need to minimize $F(\beta)$.  Under some mild assumptions about $X$ and $a$, one can show that there is a unique, optimal value of $\beta$ and that it satisfies 

$$F'(\beta) = 0$$

Show that the optimal $\beta$ satisfies 

$$\frac{\textrm{d}}{\textrm{d}\beta}\ln Z(\beta) = \mathbb{E}_{-\beta}\left[X\:\vert\: X > a\right]$$

(You can assume that all integrals involved converge uniformly, and so you are free to interchange derivatives and integrals.)

## Part 2

Under some mild assumptions about $X$, one can show that 

$$\lim_{a\to\infty}\mathbb{E}_{-\beta}\left[X\:\vert\: X > a\right] = a$$

for $\beta \geq 0$.  Combined with part 1, this means that we can approximate the optimal $\beta$ by solving 

$$\frac{\textrm{d}}{\textrm{d}\beta}\ln Z(\beta) = a$$

Use this formula to approximate the optimal value of $\beta$ when $X\sim \textrm{Exp}(1)$ and $a = 10$.  (We solved this problem in class and found that the true optimal value was $\beta \approx 0.90499$.)

(In this particular problem, you should find that solving this formula for $\beta$ is quite simple.  In general, you would typically have to resort to root-finding algorithms like `scipy.optimize.root` to solve this equation.)

# Problem 2

Suppose $X\sim\textrm{Exp}(2)$ and $Y\sim\textrm{Exp}(1)$ are independent random variables.  and define $Z = \max\{X,\, Y\}$. 

Use exponential tilting on *both* $X$ and $Y$ to estimate 

$$\mathbb{P}\left[Z > 20\right]$$

You do not have to calculate the opitmal value(s) of $\beta$, but you should give some justification for why your choice is reasonable.  

Report your estimate along with a 95% confidence interval.

In [15]:
import numpy as np

lamda1, lamda2 = 2, 1
def cdf(z):
    return (1 - np.exp(-lamda1 * z)) * (1 - np.exp(-lamda2 * z))

ans = 1 - cdf(20)
ans

2.0611535811454473e-09

In [19]:
n = 1000000
beta1 = 39/20
beta2 = 19/20

a = 20
zdelta = 1.96

# Direct method
u1 = np.random.uniform(0, 1, size=n)
u2 = np.random.uniform(0, 1, size=n)
x = -np.log(u1) / (2-beta1)
y = -np.log(u2) / (1-beta2)
z = np.maximum(x, y)

# Estimate optimal beta
print("Importance Sampling: ")

integrand = (z > a) * 2 * 20 * 20 * np.exp(- beta1 * x - beta2 * y)

theta = np.mean(integrand)
var = np.var(integrand)
error = zdelta * np.sqrt(var / n)
ci = np.array([-1, 1]) * error + theta

print(f"Theta = {theta}")
print(f"Confidence interval: {ci}")
print(f"Exact answer: {ans}")
print(f"error: {theta-ans}")



Importance Sampling: 
Theta = 2.0473503210611492e-09
Confidence interval: [1.95262720e-09 2.14207344e-09]
Exact answer: 2.0611535811454473e-09
error: -1.380326008429812e-11


# Problem 3

Suppose $X\sim N(0, 1)$ and $Y\sim N(0, 4)$ are independent random variables.  Consider the probability 

$$\theta = \mathbb{P}\left[V > 10\right]$$

where $V = X + Y$.  

## Part 1

Estimate $\theta$ using direct Monte Carlo simulation.  Make sure you use a large enough sample size to get a valid estimate.  Report your estimate along with a 95% confidence interval.


In [41]:
n = 100000000
z1 = np.random.randn(n)
z2 = np.random.randn(n)
x = z1
y = z2 * 2
v = x + y
hv = (x + y) > 10

theta = np.mean(hv)
var = np.var(hv)
error = zdelta * np.sqrt(var/n)
ci = error * np.array([-1, 1]) + theta

print(f"theta: {theta}")
print(f"var: {var}")
print(f"ci: {ci}" )


theta: 4.12e-06
var: 4.1199830256007065e-06
ci: [3.72216427e-06 4.51783573e-06]



## Part 2

Estimate $\theta$ using exponential tilting in $V$.  You can use the fact that the moment generating function of a normal random variable with mean $\mu$ and variance $\sigma^2$ is given by 

$$Z(\beta) = e^{\beta\mu + \sigma^2\beta^2/2}$$

You should approximate the optimal value of $\beta$ using the technique outlined in problem 1.  Report your estimate along with a 95% confidence interval. How much did this reduce the error from part 1? 


In [102]:
beta = 2
N = 100000000
z = np.random.randn(N)
v = np.sqrt(5) * z + 5 * beta

def Z(beta):
    return np.exp((5 * beta ** 2) / 2)

integrand = (v > 10) * np.exp(-beta * v) * Z(2)
theta = np.mean(integrand)
var = np.var(integrand)
error_ = zdelta * np.sqrt(var/n)
ci = error_ * np.array([-1, 1]) + theta
reduction = 100 * (1 - error_ / error)

print(f"theta: {theta}")
print(f"var: {var}")
print(f"ci: {ci}")
print(f"reduction: {reduction}%")


theta: 3.872545621313534e-06
var: 7.584969289236722e-11
ci: [3.87083862e-06 3.87425262e-06]
reduction: 99.57092896294353%



## Part 3

Repeat part 2, but this time approximate the optimal value of $\beta$ using the method described in lecture 15.  (I.e., approximate $F(\beta)$ and then minimize that function numerically.)  Report your estimate along with a 95% confidence interval. How much did this reduce the error from part 1? 

In [106]:
from scipy.optimize import minimize

N = 1000000
z = np.random.randn(N)
v = np.sqrt(5) * z

def Z(beta):
    return np.exp((5 * beta ** 2) / 2)

def F_hat(beta, v):
    out = (v > 10) ** 2 * np.exp(-beta * v) * Z(beta)
    return out.mean()


In [107]:
out = minimize(lambda beta: F_hat(beta, v), 0, tol=1e-10)
out

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.6138110908526286e-10
        x: [ 2.015e+00]
      nit: 21
      jac: [-4.720e-11]
 hess_inv: [[ 7.704e+08]]
     nfev: 52
     njev: 26

In [110]:
beta_hat = out.x[0]
beta_hat

2.0154628390296314

In [109]:
beta = beta_hat
N = 100000000
z = np.random.randn(N)
v = np.sqrt(5) * z + 5 * beta

integrand = (v > 10) * np.exp(-beta * v) * Z(2)
theta = np.mean(integrand)
var = np.var(integrand)
error_ = zdelta * np.sqrt(var/n)
ci = error_ * np.array([-1, 1]) + theta
reduction = 100 * (1 - error_ / error)

print(f"theta: {theta}")
print(f"var: {var}")
print(f"ci: {ci}")
print(f"reduction: {reduction}%")


theta: 3.3151853473187206e-06
var: 5.5383838232691075e-11
ci: [3.31372671e-06 3.31664399e-06]
reduction: 99.63335665058867%


# Problem 4

Consider the random variable 

$$L = \sum_{i = 1}^{m}X_i$$

where $X_i = c_i Y_i$ and the $c_i$ are positive constants and $Y_i\sim\textrm{Bern}(p_i)$ are independent (but not necessarily identically distributed).  We want to estimate 

$$\theta = \mathbb{P}[L > \ell]$$

We already did this in class when the $c_i$ and $p_i$ were all the same, but now we will look at a case where they vary with $i$.  In particular, suppose that $m = 10$ and 

$$c_i = i + 1 \:\textrm{ and }\: p_i = \frac{1}{i + 1}$$

for $i = 1, \dotsc, 10$ and $\ell = 40$.

In each part, you should implement your method with at least $100,000$ samples.  



## Part 1

Estimate $\theta$ using direct Monte Carlo simulation.  Report your estimate along with a 95% confidence interval.  (You can use the code from class as a template, so this should be easy.)


In [139]:
c = np.arange(1, 11) + 1
p = 1 / c

m = 10
l = 40

N = 1000000

loss = np.zeros(N)

for i in range(N):
    u = np.random.uniform(0, 1, m)
    loss[i] = (c @ (u < p)) > l

theta = np.mean(loss)
var = np.var(loss)
error = zdelta * np.sqrt(var/n)
ci = error * np.array([-1, 1]) + theta

print(f"theta: {theta}")
print(f"var: {var}")
print(f"ci: {ci}")

theta: 0.000737
var: 0.0007364568310000001
ci: [0.00073168 0.00074232]



## Part 2

Estimate $\theta$ using exponential tilting in each of the $X_i$'s.  Use the same value of $\beta$ for each $X_i$.  You should approximate the optimal value of $\beta$ using either the approach from problem 1 or from lecture 15.  (You can use the code from class as a template.  The hard part of this problem is finding the optimal $\beta$.)

**Hint**: The moment generating function of a sum of random variables is the product of their moment generating functions.